# Cadastre

Example montrant comment créer une géométrie sur la plateforme Oplus à partir de
données du cadastre.

Vous pouvez lancer ce notebook dans google colab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openergy/oplusclient/blob/master/samples/cadastre.ipynb)

## Installation et import des dépendances

In [ ]:
!pip install oplusclient geopandas shapely matplotlib descartes

In [ ]:
import geopandas as pd
import shapely
import requests
import io
import gzip
from oplusclient.tools import Floorplan
from oplusclient import Client
%matplotlib inline

## Récupération des données de cadastre de la commune

Changer lat et lon pour la latitude et longitude d'un point à l'intérieur du bâtiment. (à récupérer sur google maps par exemple)

In [ ]:
lat, lon = 48.835039, 2.252281
coords = [lon, lat]
resp = requests.get(f"https://geo.api.gouv.fr/communes?lon={lon}&lat={lat}").json()
if len(resp) == 0:
    raise ValueError("Ces coordonnées ne sont pas dans une commune française.")
print(f"Ville: {resp[0]['nom']}")
code_commune = resp[0]["code"]
if code_commune == "75056":
    arr = int(input("Pour Paris, veuillez donner le numéro d'arrondissement: "))
    code_commune = str(75100 + arr)
elif code_commune == "69123":
    arr = int(input("Pour Lyon, veuillez donner le numéro d'arrondissement: "))
    code_commune = str(69380 + arr)
elif code_commune == "13055":
    arr = int(input("Pour Marseille, veuillez donner le numéro d'arrondissement: "))
    code_commune = str(13200 + arr)
df = pd.read_file(io.BytesIO(gzip.decompress(requests.get(
    f"https://cadastre.data.gouv.fr/bundler/cadastre-etalab/communes/{code_commune}/geojson-gz/batiments"
).content)))
df.to_crs(epsg=2154).plot()

## Restriction aux bâtiments environnants

La variable `e` permet de déterminer dans quel périmètre on va récupérer les bâtiments alentour (pour faire des shadings).

In [ ]:
e = 0.0005
point = shapely.geometry.Point(coords)
surroundings_df = df.cx[coords[0] - e: coords[0] + e, coords[1] - e: coords[1] + e]
surroundings_df = surroundings_df.copy()
surroundings_df["shading"] = surroundings_df.geometry.map(lambda x: not point.within(x))
base = surroundings_df.plot("shading", figsize = (5,5))
pd.GeoDataFrame(geometry=pd.points_from_xy([lon], [lat])).plot(ax=base, color='r')

## Export vers oplus

### Génération du floorplan

In [ ]:
# creation d'un unique étage à partir du cadastre
floorplan = Floorplan.geo_data_frame_to_floorplan(surroundings_df, story_name="etage_0", story_height=4.5)

In [ ]:
# ajout de deux étages
for new_floor_name in ("etage_1", "etage_2"):
    # ajout de l'étage
    floorplan.add_story(new_floor_name, height=3)

    # remplissage de toutes les zones thermiques à cet étage
    for space in floorplan.json_data["stories"][0]["spaces"]:  # on parcourt toutes les zones de l'étage 0
        floorplan.copy_space_to_story(space["name"], "etage_0", new_floor_name)  # on les copie dans l'étage 1

    # remplissage de tous les shadings à cet étage (même logique)
    for shading in floorplan.json_data["stories"][0]["shading"]:
        floorplan.copy_shading_to_story(shading["name"], "etage_0", new_floor_name)

In [ ]:
# creation d'une définition de vitrage (taux de surface vitrée : 30%)
floorplan.add_window_definition("win_30", window_definition_mode="windowToWallRatio", wwr=0.3)

# application de cette définition sur toutes les parois extérieures
floorplan.add_window_to_all_exterior_edges("win_30")

### Ajout dans la plateforme

Modifier les variables organization_name, project_name, geometry_name et geometry_already_exists de manière appropriée
(Le token demandé doit préalablement avoir été récupéré [ici](https://oplus-back.openergy.fr/api/v1/oteams/users/me/api_token)).

Le projet doit avoir été créé au préalable.

In [ ]:
# transformation du floorplan en texte (pour envoi à Oplus)
floorplan_str = floorplan.save()

In [ ]:
organization_name = ""
project_name = ""
geometry_name = ""
geometry_already_exists = False

client = Client()
orga = client.get_organization(organization_name)
orga.take_seat()
project = orga.get_project(project_name)
if geometry_already_exists:
    geometry = project.get_geometry(geometry_name)
else:
    geometry = project.create_geometry(geometry_name, "floorspace")
geometry.import_file(io.StringIO(floorplan_str))
orga.leave_seat()
print(
    "La géométrie a été créée avec succès dans Oplus. Elle est visualisable ici:\n"
    f"https://oplus.openergy.fr/projects/{project.id}/geometries/{geometry.id}/viewer3D"
)
